In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder\
                    .appName('Analyzing tweet data')\
                    .getOrCreate()

In [3]:
tweets = spark.read\
              .format('json')\
              .load('fake_tweets_flat.json')

In [4]:
tweets.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- id: long (nullable = true)
 |-- in_reply_to_screen_name: string (nullable = true)
 |-- in_reply_to_status_id: long (nullable = true)
 |-- in_reply_to_user_id: long (nullable = true)
 |-- retweet_status: struct (nullable = true)
 |    |-- retweet_status_id: long (nullable = true)
 |    |-- retweet_status_text: string (nullable = true)
 |    |-- user: struct (nullable = true)
 |    |    |-- followers_count: long (nullable = true)
 |    |    |-- friends_count: long (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- screen_name: string (nullable = true)
 |    |    |-- verified: boolean (nullable = true)
 |-- text: string (nullable = true)
 |-- user: struct (nullable = true)
 |    |-- followers_count: long (nullable = true)
 |    |-- friends_count: long (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- screen_name: string (nullable = true)
 |    |-- verified: boolean (nullable = true)



In [5]:
tweets.createOrReplaceTempView('tweets')

In [6]:
tweets = spark.sql("SELECT * FROM tweets")

In [7]:
tweets.count()

1125806

In [8]:
tweets.toPandas().head()

,created_at,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,retweet_status,text,user
0,2019-02-11T18:05:26.000Z,1095021004450160641,None,NaN,NaN,"(1094783344326397952, Album That Has Nothing O...",RT @TheOnion: Album That Has Nothing On Fleetw...,"(84, 144, 3231436660, The_Aluise, False)"
1,2019-02-11T18:05:26.000Z,1095021004865372160,BreitbartNews,1.095020e+18,457984599.0,None,@BreitbartNews Probably FL at least - they’ve ...,"(1646, 839, 21609902, joseph12xu, False)"
2,2019-02-11T18:05:29.000Z,1095021017666383872,BreitbartNews,1.095020e+18,457984599.0,None,@BreitbartNews This bonehead is smoking reefer...,"(10, 53, 191896414, mk3200, False)"
3,2019-02-11T18:05:52.000Z,1095021113736863744,None,NaN,NaN,"(1095006313023787009, Survey Finds Many Gamers...",RT @TheOnion: Survey Finds Many Gamers Never F...,"(3, 44, 1070355185531305986, Ya_Boyo, False)"
4,2019-02-11T18:06:03.000Z,1095021159844904963,BreitbartNews,1.095014e+18,457984599.0,None,"@BreitbartNews Yeah, its called weather. I co...","(5562, 5648, 971194553024868354, Swesbutte, Fa..."


In [9]:
tweets_flattened = tweets.withColumn('user_followers_count', tweets.user.followers_count)\
                         .withColumn('user_friends_count', tweets.user.friends_count)\
                         .withColumn('user_id', tweets.user.id)\
                         .withColumn('user_screen_name', tweets.user.screen_name)\
                         .withColumn('user_verified', tweets.user.verified)

In [10]:
tweets_flattened = tweets_flattened.drop('user')

In [11]:
tweets_flattened.toPandas().head()

,created_at,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,retweet_status,text,user_followers_count,user_friends_count,user_id,user_screen_name,user_verified
0,2019-02-11T18:05:26.000Z,1095021004450160641,None,NaN,NaN,"(1094783344326397952, Album That Has Nothing O...",RT @TheOnion: Album That Has Nothing On Fleetw...,84,144,3231436660,The_Aluise,False
1,2019-02-11T18:05:26.000Z,1095021004865372160,BreitbartNews,1.095020e+18,457984599.0,None,@BreitbartNews Probably FL at least - they’ve ...,1646,839,21609902,joseph12xu,False
2,2019-02-11T18:05:29.000Z,1095021017666383872,BreitbartNews,1.095020e+18,457984599.0,None,@BreitbartNews This bonehead is smoking reefer...,10,53,191896414,mk3200,False
3,2019-02-11T18:05:52.000Z,1095021113736863744,None,NaN,NaN,"(1095006313023787009, Survey Finds Many Gamers...",RT @TheOnion: Survey Finds Many Gamers Never F...,3,44,1070355185531305986,Ya_Boyo,False
4,2019-02-11T18:06:03.000Z,1095021159844904963,BreitbartNews,1.095014e+18,457984599.0,None,"@BreitbartNews Yeah, its called weather. I co...",5562,5648,971194553024868354,Swesbutte,False


### Find original posts

In [12]:
tweets_flattened.createOrReplaceTempView('tweets_flattened')

In [13]:
original_posts = spark.sql(
        """
            SELECT * 
            FROM tweets_flattened 
            WHERE retweet_status IS NULL AND in_reply_to_status_id IS NULL
        """)

In [14]:
original_posts.toPandas().head()

,created_at,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,retweet_status,text,user_followers_count,user_friends_count,user_id,user_screen_name,user_verified
0,2019-02-11T18:42:33.000Z,1095030343944597509,None,None,NaN,None,Pelosi And House Leadership Demand Rep. Omar A...,68440,4027,14792049,politicususa,False
1,2019-02-11T18:44:13.000Z,1095030762628239360,None,None,NaN,None,The way this idiot defends Traitor Trump and p...,44527,39,878284831,OccupyDemocrats,False
2,2019-02-11T18:58:04.000Z,1095034250527731718,None,None,NaN,None,American People Hire High-Powered Lobbyist To ...,11028654,14,14075928,TheOnion,True
3,2019-02-11T19:02:39.000Z,1095035401255993349,TheOnion,None,14075928.0,None,@TheOnion hits another one out of the park,300,177,14631584,msiniscalchi,False
4,2019-02-11T19:12:19.000Z,1095037837211246592,BreitbartNews,None,457984599.0,None,@BreitbartNews PLEASE PAY ATTENTION-All these ...,131,156,27843452,itsnotme82,False


In [15]:
original_posts.count()

540

In [16]:
original_posts = original_posts.dropDuplicates(['id'])
original_posts.createOrReplaceTempView('original_posts')

In [17]:
original_posts_ids = spark.sql("SELECT id FROM original_posts")

In [18]:
original_posts_ids.show()

+-------------------+
|                 id|
+-------------------+
|1095076720347172865|
|1095117541448531970|
|1095428960543035393|
|1095480534170460161|
|1095704714199605248|
|1095441057297453056|
|1095495793593671681|
|1095855394994511872|
|1096054808140808192|
|1095042250176897024|
|1095206727643992064|
|1095658642576166913|
|1095730156755472385|
|1095919633436094465|
|1095321830724907008|
|1095837870428954624|
|1096084185008230400|
|1095087361925439488|
|1095111057171931136|
|1095175072984326144|
+-------------------+
only showing top 20 rows



### Find all retweets

In [19]:
retweet_posts = spark.sql(
        """
            SELECT * 
            FROM tweets_flattened 
            WHERE retweet_status IS NOT NULL
        """)

In [20]:
retweet_posts.count()

1104500

In [24]:
retweet_posts = retweet_posts.dropDuplicates(['id'])
retweet_posts.count()

70869

In [25]:
retweet_posts.createOrReplaceTempView('retweet_posts')

### Keep only the retweet posts for which we have the original tweet

In [39]:
retweet_posts = spark.sql(
    """
        SELECT * 
        FROM retweet_posts
        WHERE retweet_posts.retweet_status.retweet_status_id IN (SELECT id FROM original_posts)
    """)

In [40]:
retweet_posts.toPandas().head()

,created_at,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,retweet_status,text,user_followers_count,user_friends_count,user_id,user_screen_name,user_verified
0,2019-02-12T00:40:43.000Z,1095120478677712896,None,None,None,"(1095117541448531970, Man To Continue Slowly D...",RT @TheOnion: Man To Continue Slowly Drifting ...,169,260,3196872127,carnolld,False
1,2019-02-12T04:48:56.000Z,1095182944799318018,None,None,None,"(1095117541448531970, Man To Continue Slowly D...",RT @TheOnion: Man To Continue Slowly Drifting ...,2,22,820308956518219776,kschool713,False
2,2019-02-12T02:51:10.000Z,1095153310888284160,None,None,None,"(1095117541448531970, Man To Continue Slowly D...",RT @TheOnion: Man To Continue Slowly Drifting ...,405,1437,112917372,KoozieMan,False
3,2019-02-13T04:48:44.000Z,1095545282211659781,None,None,None,"(1095117541448531970, Man To Continue Slowly D...",RT @TheOnion: Man To Continue Slowly Drifting ...,17,54,529066803,thatsarahkeenan,False
4,2019-02-12T00:37:14.000Z,1095119601883643904,None,None,None,"(1095117541448531970, Man To Continue Slowly D...",RT @TheOnion: Man To Continue Slowly Drifting ...,274,780,245918943,zoli_bojtos,False


### Take one story

In [49]:
spark.sql("SELECT * FROM original_posts LIMIT 10").toPandas().head()

,created_at,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,retweet_status,text,user_followers_count,user_friends_count,user_id,user_screen_name,user_verified
0,2019-02-11T21:46:50.000Z,1095076720347172865,TheOnion,None,14075928.0,None,@TheOnion \n\n Scientists Reveal Everything's...,0,0,1093453828492537857,LeitgebWalter,False
1,2019-02-12T00:29:02.000Z,1095117541448531970,None,None,NaN,None,Man To Continue Slowly Drifting Into Middle Of...,11028976,14,14075928,TheOnion,True
2,2019-02-12T21:06:30.000Z,1095428960543035393,BreitbartNews,None,457984599.0,None,@BreitbartNews YOUR GREEN NEW DEAL MODE OF TRA...,124,719,339873267,cdpoling,False
3,2019-02-13T00:31:27.000Z,1095480534170460161,BreitbartNews,None,457984599.0,None,@BreitbartNews Update: NYC T-Mobile store robb...,25,91,786758315858464768,Dwightsrealnews,False
4,2019-02-13T15:22:15.000Z,1095704714199605248,None,None,NaN,None,Death Cult Democrats: Killing newborn babies t...,14872,2412,76227785,DCClothesline,False


In [84]:
spark.sql(
    """
        SELECT * 
        FROM retweet_posts 
        WHERE retweet_status.retweet_status_id = 1095704714199605248
        ORDER BY created_at
    """).count()

4

### Remove tweets with no retweets

In [85]:
spark.sql(
    """
        SELECT * FROM original_posts
        WHERE id NOT IN (SELECT retweet_posts.retweet_status.retweet_status_id FROM retweet_posts)
    """).count()

242

In [92]:
tweets_with_no_retweets = spark.sql(
    """
        SELECT * FROM original_posts
        WHERE id NOT IN (SELECT retweet_posts.retweet_status.retweet_status_id FROM retweet_posts)
    """)

In [94]:
tweets_with_no_retweets.count()

242

In [95]:
tweets_with_no_retweets.createOrReplaceTempView('tweets_with_no_retweets')

In [106]:
original_posts = spark.sql(
    """
        SELECT *
        FROM original_posts
        WHERE id NOT IN (SELECT id FROM tweets_with_no_retweets)
    """)

In [107]:
original_posts.createOrReplaceTempView('original_posts')

In [108]:
spark.sql("SELECT * FROM original_posts").toPandas().head()

,created_at,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,retweet_status,text,user_followers_count,user_friends_count,user_id,user_screen_name,user_verified
0,2019-02-12T00:29:02.000Z,1095117541448531970,None,None,None,None,Man To Continue Slowly Drifting Into Middle Of...,11028976,14,14075928,TheOnion,True
1,2019-02-13T15:22:15.000Z,1095704714199605248,None,None,None,None,Death Cult Democrats: Killing newborn babies t...,14872,2412,76227785,DCClothesline,False
2,2019-02-14T01:21:00.000Z,1095855394994511872,None,None,None,None,David Corn says Paul Manafort's recently revea...,68444,4027,14792049,politicususa,False
3,2019-02-12T06:23:26.000Z,1095206727643992064,None,None,None,None,South Carolina: Man Converts To Islam – Plants...,14867,2413,76227785,DCClothesline,False
4,2019-02-14T00:11:22.000Z,1095837870428954624,None,None,None,None,Montana Couple Has Ranch Taken Without Due Pro...,14871,2412,76227785,DCClothesline,False


In [109]:
spark.sql(
    """
        SELECT * 
        FROM retweet_posts 
        WHERE retweet_status.retweet_status_id = 1095117541448531970
        ORDER BY created_at
    """).count()

163

In [110]:
spark.sql(
    """
        SELECT * 
        FROM retweet_posts 
        WHERE retweet_status.retweet_status_id = 1095704714199605248
        ORDER BY created_at
    """).count()

4

In [111]:
spark.sql(
    """
        SELECT * 
        FROM retweet_posts 
        WHERE retweet_status.retweet_status_id = 1095855394994511872
        ORDER BY created_at
    """).count()

84

In [112]:
spark.sql(
    """
        SELECT * 
        FROM retweet_posts 
        WHERE retweet_status.retweet_status_id = 1095206727643992064
        ORDER BY created_at
    """).count()

2

In [113]:
spark.sql(
    """
        SELECT * 
        FROM retweet_posts 
        WHERE retweet_status.retweet_status_id = 1095837870428954624
        ORDER BY created_at
    """).count()

1

In [114]:
original_posts.write.json('original_posts.json')

In [118]:
original_posts.coalesce(1).write.format('json').save('original_posts.json')

In [119]:
retweet_posts.coalesce(1).write.format('json').save('retweet_posts.json')